# Named Entity Recognition

Two different Natural Language Processing libraries are going to be applied for the purpose of identifying entities in out dataset (considering as dataset the questions extracted from the LC-QuAD dataset and the corresponding entities for each question as the real labels).

## Flair

In [1]:
#import sys
#!{sys.executable} -m pip install flair

from flair.data import Sentence
from flair.models import SequenceTagger

tagger = SequenceTagger.load('ner')
tagger2 = SequenceTagger.load('ner-ontonotes')
tagger3 = SequenceTagger.load('por')
tagger4 = SequenceTagger.load('frame')

sentence = Sentence('George went to Washington .')

# predict NER tags
tagger.predict(sentence)

# print sentence with predicted tags
print(sentence.to_tagged_string())

tagger2.predict(sentence)
print(sentence.to_tagged_string())

C:\Users\agui\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


George <S-PER> went to Washington <S-LOC> .
George <S-PERSON> went to Washington <S-GPE> .


In [ ]:
import re

reg1 = re.compile(r' <B-\w+> ')
reg2 = re.compile(r' <E-\w+>')
reg3 = re.compile(r' <I-\w+> ')


#ofile = open('results/NER.txt', 'wt', encoding='utf-8')

tags = []

with open('sqa_questions_index.txt', 'rt', encoding='utf-8') as f:
    questions = f.read().splitlines()

for question in questions:
    idx, question = question.split(',', 1)
    sentence = Sentence(question.replace('?', '').strip())
    # predict NER tags
    tagger.predict(sentence)
        
    for entity in sentence.get_spans('ner'):
        tag, _ = str(entity).split('-',1)
        if tag not in tags:
            tags.append(tag)
        _, ent = str(entity).split(']: \"')
        ent = ent.replace('\"', '')
        #ofile.writelines(idx + ',' + ent +'\n')
        
print(tags)        
#ofile.close()

In [ ]:
### S - Single,   B - Begin,   E - End,   I - Intermediate
### PER: Person,    LOC: Location,    ORG: Organization,    MISC: Miscelaneous

In [ ]:
import re

reg1 = re.compile(r' <B-\w+> ')
reg2 = re.compile(r' <E-\w+>')
reg3 = re.compile(r' <I-\w+> ')


#ofile = open('results/NER12.txt', 'wt', encoding='utf-8')

tags = []

with open('sqa_questions_index.txt', 'rt', encoding='utf-8') as f:
    questions = f.read().splitlines()

for question in questions:
    idx, question = question.split(',', 1)
    sentence = Sentence(question.replace('?', '').strip())
    # predict NER tags
    tagger2.predict(sentence)
        
    for entity in sentence.get_spans('ner'):
        tag, _ = str(entity).split('-',1)
        if tag not in tags:
            tags.append(tag)
        _, ent = str(entity).split(']: \"')
        ent = ent.replace('\"', '')
        #ofile.writelines(idx + ',' + ent +'\n')
        
print(tags)        
#ofile.close()

In [3]:
### both tags together

import re

reg1 = re.compile(r' <B-\w+> ')
reg2 = re.compile(r' <E-\w+>')
reg3 = re.compile(r' <I-\w+> ')


ofile1 = open('results/NER_tags_2.txt', 'wt', encoding='utf-8')
ofile2 = open('results/NER12_tags_2.txt', 'wt', encoding='utf-8')

rem_tags = ['CARDINAL','ORDINAL','DATE']
# We exclude these types to exclude wrong identification of entities in the questions
# Example: 514,What is the agency name of the Ministry of Information and Communication Technology (Thailand)
#          and distribution region  of the Channel 9 MCOT HD ?
# In this question, '9' would be identified as a cardinal entity, but the real entity in this case is 'Channel 9 MCOT HD',
# so excluding the cardinal type we allow the model to identify the full compound entity.

with open('sqa_questions_index.txt', 'rt', encoding='utf-8') as f:
    questions = f.read().splitlines()

remove = ['What is ', 'What are ', 'Which ', 'Who is ', 'Who were ', 'Who ', 'Was ', 'How many ', 'How much ',
              'has been ', 'Do ', 'Does ', 'How ', 'Where ', 'go ']    

questions_cleaned = []

for question in questions:
    for rem in remove:
        if rem in question:
            question = question.replace(rem,'')

    questions_cleaned.append(question)

for question in questions_cleaned:
    idx, question = question.split(',', 1)
    try:
        sentence = Sentence(question.replace('?', '').replace('Name ','').strip())
        sentence2 = Sentence(question.replace('?', '').replace('Name ','').strip())
    except:
        print(question)
    # predict NER tags
    tagger.predict(sentence)
    tagger2.predict(sentence2)
        
    for entity in sentence.get_spans('ner'):
        tag, _ = str(entity).split('-',1)
        _, ent = str(entity).split(']: \"')
        ent = ent.replace('\"', '')
        ofile1.writelines(idx + ',' + ent +'\n')           
            
    for entity in sentence2.get_spans('ner'):
        tag, _ = str(entity).split('-',1)
        if tag not in rem_tags:
            _, ent = str(entity).split(']: \"')
            ent = ent.replace('\"', '')
            ofile2.writelines(idx + ',' + ent +'\n')

ofile1.close()
ofile2.close()

In [47]:
with open('results/NER_tags_2.txt', 'rt', encoding='utf-8') as f:
    entities_ner = f.read().splitlines()
    
with open('results/NER12_tags_2.txt', 'rt', encoding='utf-8') as f:
    entities_ner12 = f.read().splitlines()

In [48]:
print(entities_ner[0:10])
print(entities_ner12[0:10])

['0,Bill Finger', '1,winston churchill', '1,Selwyn Lloyd', '2,Gestapo', '3,Mumbai North', '4,Roberto Clemente Bridge', '5,Theos Philopator', '5,Cleopatra', '6,Li Si', '7,PAvel Moroz']
['0,Bill Finger', '1,winston churchill', '1,Selwyn Lloyd', '2,Gestapo', '3,Mumbai North', '4,Roberto Clemente Bridge', '6,Li Si', '7,PAvel Moroz', '7,Yakov Estrin', '8,Broadmeadows, Victoria']


In [52]:
n = 5000

ner = [['']]*n
ner12 = [['']]*n

# store the entities identified by each Flair NER model
for entity in entities_ner:
    idx, ent = entity.split(',', 1)
    if ner[int(idx)] == ['']:
        ner[int(idx)] = [entity]
    else:
        ner[int(idx)].append(entity)
        
for entity in entities_ner12:
    idx, ent = entity.split(',', 1)
    entity = entity.replace('the ','')
    if ner12[int(idx)] == ['']:
        ner12[int(idx)] = [entity]
    else:
        ner12[int(idx)].append(entity)

In [53]:
ner[0:20]

[['0,Bill Finger'],
 ['1,winston churchill', '1,Selwyn Lloyd'],
 ['2,Gestapo'],
 ['3,Mumbai North'],
 ['4,Roberto Clemente Bridge'],
 ['5,Theos Philopator', '5,Cleopatra'],
 ['6,Li Si'],
 ['7,PAvel Moroz', '7,Yakov Estrin'],
 ['8,Broadmeadows, Victoria'],
 ['9,ASC Creative Services'],
 ['10,Ernest Rutherford', '10,Charles Drummond Ellis'],
 ['11,REP Parasol'],
 ["12,Monroe Carell Jr. Children's Hospital",
  '12,Vanderbilt',
  '12,Duncan U. Fletcher'],
 ['13,Nader Guirat,', '13,Josef Johansson'],
 [''],
 ['15,HBO'],
 ['16,Nikolai Morozov', '16,Stanislav Morozov'],
 ['17,Abhijit Kunte', '17,Kasparov'],
 ['18,MSX Basics'],
 ['19,Ronaldo']]

In [54]:
ner12[0:20]

[['0,Bill Finger'],
 ['1,winston churchill', '1,Selwyn Lloyd'],
 ['2,Gestapo'],
 ['3,Mumbai North'],
 ['4,Roberto Clemente Bridge'],
 [''],
 ['6,Li Si'],
 ['7,PAvel Moroz', '7,Yakov Estrin'],
 ['8,Broadmeadows, Victoria'],
 ['9,ASC Creative Services'],
 ['10,Ernest Rutherford', '10,Charles Drummond Ellis'],
 ['11,REP Parasol'],
 ["12,Monroe Carell Jr. Children's Hospital",
  '12,Vanderbilt',
  '12,Duncan U. Fletcher'],
 ['13,Nader Guirat,', '13,Josef Johansson'],
 ['14,sarah jane'],
 ['15,HBO'],
 ['16,Nikolai Morozov', '16,Stanislav Morozov'],
 ['17,Abhijit Kunte', '17,Kasparov'],
 [''],
 ['19,Ronaldo']]

In [57]:
ner_ner12[0:20]

[['0,Bill Finger'],
 ['1,winston churchill', '1,Selwyn Lloyd'],
 ['2,Gestapo'],
 ['3,Mumbai North'],
 ['4,Roberto Clemente Bridge'],
 ['5,Theos Philopator', '5,Cleopatra'],
 ['6,Li Si'],
 ['7,PAvel Moroz', '7,Yakov Estrin'],
 ['8,Broadmeadows, Victoria'],
 ['9,ASC Creative Services'],
 ['10,Ernest Rutherford', '10,Charles Drummond Ellis'],
 ['11,REP Parasol'],
 ["12,Monroe Carell Jr. Children's Hospital",
  '12,Vanderbilt',
  '12,Duncan U. Fletcher'],
 ['13,Nader Guirat,', '13,Josef Johansson'],
 ['14,sarah jane'],
 ['15,HBO'],
 ['16,Nikolai Morozov', '16,Stanislav Morozov'],
 ['17,Abhijit Kunte', '17,Kasparov'],
 [''],
 ['19,Ronaldo']]

In [56]:
#combine the entities identified by each model
ner_ner12 = ['']*n

for i in range(0, n):
    if len(ner[i])==len(ner12[i]):
        if all(elem in ner12[i] for elem in ner[i]) == True:   # when both models identify the same entities
            ner_ner12[i] = ner[i]
        else:
            if ner[i] == ['']:     # when first model doesn't identify any entitiy but the second does
                ner_ner12[i] = ner12[i]            
            else:
                inter = list(set(ner[i]) & set(ner12[i]))
                dif = list(set(ner12[i]) - set(ner[i]))
                ner_ner12[i] = inter+dif          
    elif len(ner[i])<len(ner12[i]):
        if all(elem in ner12[i] for elem in ner[i]) == True:     # when second model identifies more entities and the ones
            ner_ner12[i] = ner12[i]                              # identified by the first model are included there
        else:
            if ner[i] == ['']:
                ner_ner12[i] = ner12[i]            
            else:
                inter = list(set(ner[i]) & set(ner12[i]))
                dif = list(set(ner[i]) - set(ner12[i]))
                ner_ner12[i] = inter+dif
                # when some entity is compound by more than one word, it might happen that one of the models identifies it as 
                # separate entities, so this is done to just consider the longer one, which is the actual entity.
                # Example: 45,Which shareholder of Dagenham wind turbines is also the parent company of the Ford Falcon Cobra?
                # In this question the ner model identifies as entities "Dagenham" and "Ford Falcon Cobra", but the ner12
                # identifies as entities "Dagenham", "Ford" and "Falcon Cobra", which is not correct in this case. This way,
                # we exclude the entities "Ford" and "Falcon Cobra", because they are included in the entity "Ford Falcon Cobra".
                
    elif len(ner[i])>len(ner12[i]):
        if all(elem in ner[i] for elem in ner12[i]) == True:
            ner_ner12[i] = ner[i]
        else:
            if ner12[i] == ['']:
                ner_ner12[i] = ner[i]
            else:
                inter = list(set(ner[i]) & set(ner12[i]))
                dif = list(set(ner12[i]) - set(ner[i]))
                ner_ner12[i] = inter+dif       

In [13]:
with open('sqa_entities_labels_index.txt', 'rt', encoding='utf-8') as f:
    real_entities = f.read().replace('\ufeff', '').splitlines()

real = [['']]*n

for entity in real_entities:
    idx, ent = entity.split(',', 1)
    if real[int(idx)] == ['']:
        real[int(idx)] = [entity]
    else:
        real[int(idx)].append(entity)

In [14]:
real[0:10]

[['0,Bill Finger'],
 ['1,Selwyn Lloyd', '1,Winston Churchill'],
 ['2,Gestapo'],
 ['3,Mumbai North'],
 ['4,Roberto Clemente Bridge'],
 ['5,Ptolemy XIII Theos Philopator', '5,Cleopatra V'],
 ['6,Li Si'],
 ['7,Pavel Moroz', '7,Yakov Estrin'],
 ['8,Broadmeadows, Victoria'],
 ['9,ASC Creative Services']]

In [15]:
print(len(ner))
print(len(ner12))
print(len(real))

5000
5000
5000


In [16]:
def compare_predictions(real, ner_pred):
    match = 0
    partial = 0
    less = 0
    more = 0
    none_match = 0
    none_less = 0
    none_more = 0
    predicted = 0

    nonpredicted = []
    same_true = []
    same_pred = []


    from sklearn.metrics import f1_score

    for i in range(0,n):
        true = [x.lower() for x in real[i]]
        pred = [x.lower() for x in ner_pred[i]]
        lt = len(true)
        lp = len(pred)

        if lt == lp:
            matching = [p for p in pred if p in true]
            if matching == pred or matching == true:
                match+=1
                predicted+=len(pred)
            else:
                if matching != []:
                    partial+=1
                    predicted+=len(pred)
                else:
                    none_match+=1
                    nonpredicted.append(pred)               

            same_true.append(true)
            same_pred.append(pred)

        elif lt > lp:
            matching = [p for p in pred if p in true]
            if matching != []:
                less+=1
                predicted+=len(pred)
            else:
                none_less+=1
                nonpredicted.append(pred)
        else:
            matching = [p for p in pred if p in true]
            if matching != []:
                more+=1
                predicted+=len(true)
            else:
                none_more+=1
                nonpredicted.append(pred)
                
    return match, partial, less, more, none_match, none_less, none_more, predicted

In [17]:
### compare only ner predictions
match, partial, less, more, none_match, none_less, none_more, predicted = compare_predictions(real, ner)
print(match)
print(partial)
print(less)
print(more)
print(none_match)
print(none_less)
print(none_more)
print(predicted)
print(predicted/len(real_entities))

3199
337
158
132
948
116
110
5122
0.7711532670882264


In [18]:
### compare only ner12 predictions
match, partial, less, more, none_match, none_less, none_more, predicted = compare_predictions(real, ner12)
print(match)
print(partial)
print(less)
print(more)
print(none_match)
print(none_less)
print(none_more)
print(predicted)
print(predicted/len(real_entities))

3034
262
191
151
1043
150
169
4886
0.7356218006624511


In [19]:
### compare ner+ner12 combined predictions

match, partial, less, more, none_match, none_less, none_more, predicted = compare_predictions(real, ner_ner12)
print(match)
print(partial)
print(less)
print(more)
print(none_match)
print(none_less)
print(none_more)
print(predicted)
print(predicted/len(real_entities))

3172
315
116
130
1047
133
87
5075
0.7640770852152966


## spaCy

In [46]:
# !{sys.executable} -m pip install spacy download en_core_web_md

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm, en_core_web_md
nlp = en_core_web_sm.load()
nlp2 = en_core_web_md.load()

In [51]:
with open('sqa_questions_index.txt', 'rt', encoding='utf-8') as f:
    questions = f.read().splitlines()


ofile = open('results/NER_spacy_2.txt', 'wt', encoding='utf-8')

remove = ['What is ', 'What are ', 'Which ', 'Who is ', 'Who were', 'Who ', 'Was ', 'How many ', 'How much ',
          'has been ', 'Do ', 'Does ', 'How ', 'Where ', 'Name ']


questions_cleaned = []

for question in questions:
    for rem in remove:
        if rem in question:
            question = question.replace(rem,'')
            
    questions_cleaned.append(question)    
    
entities = []
    
for question in questions_cleaned:    
    idx, question = question.split(',',1)
    doc = nlp2(question.replace('?', '').strip())
    [entities.append(ent.text) for ent in doc.ents]
    try:    
        for entity in entities:
            ofile.writelines(idx + ',' + entity +'\n')
    except:
        ofile.writelines(idx + ',' +'\n')
    entities = []
    
ofile.close()

0 ['Bill Finger']
1 ['Selwyn Lloyd']
2 ['Gestapo']
3 ['Mumbai']
4 ['Roberto Clemente Bridge']
5 ['ptolemy XIII Theos Philopator', 'Cleopatra V']
6 ['Li Si']
7 ['PAvel Moroz', 'Yakov Estrin']
8 ['Broadmeadows, Victoria']
9 ['ASC Creative Services']
10 ['Ernest Rutherford', 'Charles Drummond Ellis']
11 ['REP']
12 ["the Monroe Carell Jr. Children's Hospital", 'Vanderbilt', 'the Duncan U. Fletcher']
13 ['Nader Guirat', 'Josef Johansson']
14 []
15 ['HBO']
16 ['Nikolai Morozov', 'Stanislav Morozov']
17 ['Abhijit Kunte', 'Kasparov']
18 []
19 ['seasons', 'Ronaldo']
20 ['SamurAbsheron', 'Anar Salmanov']
21 ['2nd Foreign Infantry Regiment']
22 ['3Sub']
23 ['Kevin Jonas', 'Joe Jonas']
24 ['Timm Gunn', 'Sunrise', 'HIMYM']
25 ['O.co Coliseum']
26 ['Rishkiesh']
27 []
28 ['PhDs', 'National Medal of Science']
29 ['Aston Villa 2000-02 season', 'Middlesbrough F.C.', '2009']
30 ['William Harper']
31 ['PCL']
32 ['Kevin Jonas', 'Jonas']
33 ['Sagan']
34 ['Rosemont', 'Illinois']
35 ['Greenup County High Scho

626 ['Rodrigues']
627 ['Waxiang']
628 ['Bogdanua', 'Upper Neretva']
629 ['Charles Drummond Ellis', 'PhD']
630 ['Chevrolet']
631 ['Martin Pugh', 'Alana Stewert']
632 ['the Pepsi Center']
633 []
634 ['US', 'Alfred Phillips', 'William Smith']
635 ['Michael Jeffery', 'Williuam Deane']
636 ['Richard Winn']
637 ['Neil Brown']
638 []
639 ['Mount Hoffmann']
640 ['Agalga']
641 ['LG Optimus Vu', 'Manu Cornet']
642 ['Edsel']
643 ['Menora']
644 ['Cape Town', 'the Royal Astronomical Society']
645 ['John McEwen', 'Phm Vn']
646 []
647 []
648 ['the Caspian Sea']
649 ['Congress of Industrial Organizations']
650 ['Catarina Lindgren']
651 ['John of Damascus']
652 ['the West Berkshire Council']
653 ['Hank Williams Jr.']
654 ['Pixar Canada', 'Pixar']
655 ['Indian']
656 ['NYC FC']
657 ['Tony Bennett']
658 ['English']
659 ['Sarah Jane', 'Ron Grainer']
660 ['David Isaacs']
661 []
662 ['Brazil', '2']
663 ['Tirana']
664 ['Jon', 'Vangelis']
665 []
666 ['Marika Gombitov']
667 ['9TV']
668 ['the Army Air Corps', 'I

1256 ['South']
1257 ['Battle of Fort stephenson']
1258 ['the American Mediterranean Sea', 'Entronque de Herradura']
1259 ["the Brown's Corners", 'Toronto', 'the George Karrys']
1260 ['Alcal de Henares']
1261 ['Guatemala']
1262 []
1263 ['Provisional Government of Saskatchewan']
1264 ['The Farmer Wants a Wife']
1265 ['Kentucky']
1266 ['US']
1267 []
1268 ['Jon Curran']
1269 ['Vietnam Airlines']
1270 ['Dartington College of Arts']
1271 ['New Yorker']
1272 ['Comcast SportsNet']
1273 []
1274 []
1275 ['PopCon', 'Louis Le Cocqq']
1276 ['Cyrus Mann']
1277 ['William Cushing']
1278 ["Filbert's Old Time Root Beer"]
1279 ['Primus', 'Comedy Central']
1280 ['Stars']
1281 []
1282 ['International Union of Anarchists']
1283 ['Hugh Hefner']
1284 ['WGBC']
1285 []
1286 ['Lancy', 'Geneva']
1287 []
1288 ['Ringgold High School', 'Pennsylvania']
1289 []
1290 ['Miguel de Cervantes', 'Francisco de Robles']
1291 ['Sino French War', 'Raoul Magrin-Vernerey']
1292 ['Honeybuns']
1293 ['Willis Sharpe Kilmer']
1294 ['L

1853 ['John Byrne', 'Emma Frost']
1854 ['Chinook', 'Dennis Muilenburg']
1855 ['National League']
1856 []
1857 ['Miluk', 'Cayuse']
1858 ['AMD']
1859 ['David Charles']
1860 ['Steven Palazao']
1861 ['Stephen E. Ambrose', 'HBO']
1862 ['Volos']
1863 ['Huey, Dewey', 'Louie', 'American', 'Pekin']
1864 ['Bobby Skinstad']
1865 ['marquette Golden Eagles']
1866 ['Hiroyuki Takei']
1867 ['Jon Speelman', 'Eric Schiller']
1868 ['the Croix de Guerre']
1869 ['Flats Painted Caves']
1870 ['Islam']
1871 ['Stoke City F.C.']
1872 []
1873 ['Jimmy Wales']
1874 ['Pennsylvania']
1875 ['Arkansas']
1876 ['Ted Osborne']
1877 ['Jon Voight']
1878 ['england', '20']
1879 ['RENFE', 'Renaissance Center']
1880 ['Jim Kiick', 'New Jersey']
1881 ['the United States']
1882 ['Maple Leaf S&E']
1883 ['The Boy is Mine', 'Atlanta']
1884 ['Snake Island']
1885 ['UEM']
1886 []
1887 ['Mount Raimer']
1888 ['Tambourine Man']
1889 ['Peter Thorneycroft']
1890 []
1891 ['the Marina Pankova', 'Viktor Ulyanich']
1892 ['Michigan', 'Baltimore'

2449 ['Sierra del Merendin']
2450 []
2451 ['Hynief']
2452 ['Jared Bernstein']
2453 ['France']
2454 ['Populous']
2455 ['Ludovic Boi', 'Fernando Augustin']
2456 ['one', 'Ankh-Morpork City Watch']
2457 ['Bryan Mantia']
2458 ['Cancun']
2459 ['Seguin', 'Texas']
2460 ['Mississippi']
2461 ['Jimmy Diggs']
2462 ['Washington, D.C.']
2463 ['Siege of Fort Recovery', 'Indian']
2464 ['Stanford', '1906-1917']
2465 ['Polytech', 'Bataan', 'Taguig']
2466 ['Kelly Osbourne', 'Ozzy Osbourne']
2467 []
2468 []
2469 ['Clinton', 'Winston Bryant']
2470 []
2471 ['Boeing', 'F/A-18E', 'Dennis Muilenburg']
2472 ['cyril hume']
2473 []
2474 ['Texas', 'Dallas']
2475 []
2476 ['Nandurbar']
2477 ['Tim Burton', 'Danny Elfman']
2478 ['Jason Maas']
2479 ['William Clark']
2480 ['LATV']
2481 ['Houston Astros']
2482 ['Islamic', 'Craig Van Tilbury']
2483 ['Solidus']
2484 []
2485 ['VMF-218', 'VMF-155']
2486 ['Michael springer']
2487 ['Thud']
2488 ['Steve Nash', 'Vancouver Whitecaps FC']
2489 ['A Todo Romantic Style']
2490 ['Chri

3046 ['James Roberts', 'US']
3047 []
3048 ['Arlington Heights', 'Illinois']
3049 ['Gymnastics', '2008']
3050 ['the American Magazine', 'New York Post']
3051 ['the The Conglomerate (American group', 'Blackstreet & Ma']
3052 ['SAGE Publications']
3053 ['Demis Roussos']
3054 ['Frampton', 'Camel']
3055 ['Pittsburgh Line']
3056 ['Gerard Marino']
3057 ['260']
3058 ['Raa', 'Adriatic']
3059 ['the Clinton Foundation']
3060 []
3061 []
3062 ['the Harry Parker', 'the Carl Sitton']
3063 ['Valaichchenai']
3064 ['McClain', 'one']
3065 ['Roger Barton']
3066 ['Kala Pani']
3067 ['Timothy Morton', 'Buddhism']
3068 ['Ptolemy XII Auletes', 'Ptolemy XIII Theos Philopator']
3069 ['Kalpan', 'Inc.', 'Iowa']
3070 ['Englewood', 'Colorado', 'one']
3071 ['Shelby County', 'Tennessee']
3072 ['the South Side Elevated Railroad', 'Sakari Suzuki']
3073 ['John Roberts']
3074 []
3075 ['one', 'Andrea']
3076 ['the Connecticut Lakes']
3077 ['Blayse']
3078 ['BBC HD']
3079 ['Bacillales', 'Firmicutes']
3080 ['the National Comme

3644 ['Tampa', 'Florida']
3645 ['GetTV']
3646 ['Novair International Airways']
3647 ['the Mediterranean Basketball Association']
3648 ['Philippines']
3649 ['the US Department']
3650 ['Frasier']
3651 ['John of Damascus', 'Edwin', 'Northumbria']
3652 ['Josh Bell']
3653 ['Fort Gadsden', 'Fort Barrancas']
3654 ['Royal Thai Army']
3655 ['Haqqi al-Azm']
3656 ['sweden']
3657 ['Dolley Madison', 'Montpelier']
3658 ['Moscow']
3659 ['Latania']
3660 ['8.1', 'OS', 'Lumia', '830']
3661 ['2005']
3662 []
3663 ['Mac OS', 'C++']
3664 ['moscow']
3665 ['Trevor Peacock', 'Fred Claus']
3666 ['Jan Paulsen']
3667 []
3668 ['Saint Petersburg']
3669 []
3670 ['HBO']
3671 ['Warren Magnuson']
3672 ['Andrew Lincoln', 'Million Dollar']
3673 ['Blytheville High School']
3674 ['Budapest']
3675 ['2012', 'Campeonato Brasileiro Srie A.']
3676 ['Eddie Bravo', 'Alan Jouban']
3677 ['MSX BASIC']
3678 ['Swedish']
3679 ['Harry', 'Potters']
3680 ['Lake Uniamsi']
3681 ['Peter Cosgrove']
3682 ['Indianapolis', 'Colts', 'Carolina Pan

4245 ['Douglas Netter']
4246 ['Upper Neratva']
4247 ['Johnny Cash', 'Kris Kristofferson']
4248 ['the Journal of Cerebral Blood Flow and Metabolism']
4249 ['Elie Wiesel']
4250 ['Kenya']
4251 []
4252 ['the National Museum of Racing and Hall of Fame']
4253 ['Charing Cross Bridge', 'Dorothea Sharp']
4254 ['Israel', 'Jaber Al-Ahmad Al-Sabah']
4255 ['the Martin Ragaway', 'Chuck Connors']
4256 ['Ashanti']
4257 ['Ricky Grevais']
4258 ['03', 'Bonnie & Clyde']
4259 ['Playtone']
4260 ['Malaysia']
4261 ['Galician', 'German']
4262 []
4263 ['Wikitionary']
4264 ['Thames Estuary', 'Reading Berkshire']
4265 ['Samuel Moreno Rojas']
4266 ['Nintendo eShop', 'Nintendo Network']
4267 ["Monty Python's", 'Cinema International Corporation']
4268 ['Trinity School', 'Brentwood']
4269 ['Abiye', 'Abebe', 'the Amsale Aberra']
4270 []
4271 ['American', 'Mediterranean Sea']
4272 ['Uni of California']
4273 ['PrivateCore']
4274 ['Season 11']
4275 ['Waddy']
4276 ['Baker', 'Dozen']
4277 []
4278 ['Mike', 'Toronto']
4279 [

4817 ['Terminator', 'Terminator 2:', 'Judgment Day']
4818 ['Christen-Democratisch en Vlaams']
4819 ['Juno']
4820 ['PlayN']
4821 ['Steven Moffat']
4822 ['BBC Two']
4823 ['Tupelo', 'Mississippi']
4824 ['Walt Disney Records']
4825 ['cohoes', 'NY']
4826 ['the Columbia University', 'William Campbell']
4827 ['Maharashtran']
4828 ['Cinema International Corporation']
4829 ['Ashot Grashi']
4830 ['Ae Fond Kiss', 'Halloween']
4831 ['Steve Buyer']
4832 ['Steven Moffat']
4833 ['Michael Schumacher', 'Pole', '1994', 'Spanish']
4834 ['Shaheed Minar', 'Kolkata']
4835 ['Canada']
4836 ['Batman Live']
4837 ['San Salvador Island', 'Bahamas']
4838 ['Audi India Correct Question']
4839 ['NASA', 'Gemini']
4840 ['South Side', '400']
4841 ['Michelle D. Johnson', 'United States Air Force Academy']
4842 ['Pittsburgh']
4843 ['Stanlee', 'the Tales of Suspense']
4844 ['Middlesbrough FC']
4845 ['National Basketball Association']
4846 ['Cubic Hermite']
4847 ['Dallas']
4848 ['bloomington']
4849 ['Paradise']
4850 ['Cople

In [14]:
with open('results/NER_spacy.txt', 'rt', encoding='utf-8') as f:
    entities_spacy_sm = f.read().splitlines()
    
with open('results/NER_spacy_2.txt', 'rt', encoding='utf-8') as f:
    entities_spacy_md = f.read().splitlines()

In [15]:
n = 5000

ner_sm = [['']]*n
ner_md = [['']]*n

for entity in entities_spacy_sm:
    idx, ent = entity.split(',', 1)
    if ner_sm[int(idx)] == ['']:
        ner_sm[int(idx)] = [entity]
    else:
        ner_sm[int(idx)].append(entity)
        
for entity in entities_spacy_md:
    idx, ent = entity.split(',', 1)
    if ner_md[int(idx)] == ['']:
        ner_md[int(idx)] = [entity]
    else:
        ner_md[int(idx)].append(entity)

In [16]:
print(ner_sm[0:10])
print(ner_md[0:10])

[['0,Bill Finger'], ['1,Selwyn Lloyd'], [''], ['3,Mumbai'], ['4,Roberto Clemente Bridge'], ['5,XIII Theos Philopator', '5,Cleopatra V'], ['6,Li Si'], ['7,PAvel Moroz', '7,the Yakov Estrin'], ['8,Broadmeadows', '8,Victoria'], ['9,ASC Creative Services']]
[['0,Bill Finger'], ['1,Selwyn Lloyd'], ['2,Gestapo'], ['3,Mumbai'], ['4,Roberto Clemente Bridge'], ['5,ptolemy XIII Theos Philopator', '5,Cleopatra V'], ['6,Li Si'], ['7,PAvel Moroz', '7,Yakov Estrin'], ['8,Broadmeadows, Victoria'], ['9,ASC Creative Services']]


In [18]:
ner_sm_md = ['']*n

for i in range(0, n):
    if len(ner_sm[i])==len(ner_md[i]):
        if all(elem in ner_md[i] for elem in ner_sm[i]) == True:
            ner_sm_md[i] = ner_sm[i]
        else:
            if ner_sm[i] == ['']:
                ner_sm_md[i] = ner_md[i]            
            else:
                ner_sm_md[i] = ner_sm[i]            
    elif len(ner_sm[i])<len(ner_md[i]):
        if all(elem in ner_md[i] for elem in ner_sm[i]) == True:
            ner_sm_md[i] = ner_md[i]           
        else:
            if ner_sm[i] == ['']:
                ner_sm_md[i] = ner_md[i]            
            else:
                inter = list(set(ner_sm[i]) & set(ner_md[i]))
                dif = list(set(ner_sm[i]) - set(ner_md[i]))
                ner_sm_md[i] = inter+dif
                
    elif len(ner_sm[i])>len(ner_md[i]):
        if all(elem in ner_sm[i] for elem in ner_md[i]) == True:
            ner_sm_md[i] = ner_md[i]
        else:
            if ner_md[i] == ['']:
                ner_sm_md[i] = ner_sm[i]
            else:
                inter = list(set(ner_sm[i]) & set(ner_md[i]))
                dif = list(set(ner_md[i]) - set(ner_sm[i]))
                ner_sm_md[i] = inter+dif

[['0,Bill Finger'], ['1,Selwyn Lloyd'], ['2,Gestapo'], ['3,Mumbai'], ['4,Roberto Clemente Bridge'], ['5,XIII Theos Philopator', '5,Cleopatra V'], ['6,Li Si'], ['7,PAvel Moroz', '7,the Yakov Estrin'], ['8,Broadmeadows, Victoria'], ['9,ASC Creative Services'], ['10,Ernest Rutherford', '10,Charles Drummond Ellis'], ['11,REP Parasol'], ["12,the Monroe Carell Jr. Children's Hospital", '12,Vanderbilt', '12,the Duncan U. Fletcher'], ['13,Nader Guirat', '13,Josef Johansson'], [''], ['15,HBO'], ['16,Nikolai Morozov', '16,Stanislav Morozov'], ['17,Abhijit Kunte', '17,Kasparov'], ['18,the MSX Basics'], ['19,seasons', '19,Ronaldo'], ['20,SamurAbsheron', '20,Anar Salmanov'], ['21,2nd Foreign Infantry Regiment'], ['22,3Sub'], ['23,Kevin Jonas', '23,Joe Jonas'], ['24,Timm Gunn', '24,Sunrise (HIMYM'], ['25,O.co Coliseum'], ['26,Rishkiesh'], ['27,one'], ['28,PhDs', '28,National Medal of Science'], ['29,Aston Villa 2000-02 season', '29,Middlesbrough F.C.', '29,2009-10'], ['30,William Harper'], ['31,PCL'

In [22]:
### compare only sm predictions

match, partial, less, more, none_match, none_less, none_more, predicted = compare_predictions(real, ner_sm)
print(match)
print(partial)
print(less)
print(more)
print(none_match)
print(none_less)
print(none_more)
print(predicted)
print(predicted/len(real_entities))

TRUE:  ['0,bill finger']
PRED:  ['0,bill finger']
SAME:  ['0,bill finger']
TRUE:  ['1,selwyn lloyd', '1,winston churchill']
PRED:  ['1,selwyn lloyd']
LESS:  ['1,selwyn lloyd']
TRUE:  ['2,gestapo']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['2,gestapo']
NONE_PRED_EQUAL:  ['']
TRUE:  ['3,mumbai north']
PRED:  ['3,mumbai']
SAME:  []
NONE_TRUE_EQUAL:  ['3,mumbai north']
NONE_PRED_EQUAL:  ['3,mumbai']
TRUE:  ['4,roberto clemente bridge']
PRED:  ['4,roberto clemente bridge']
SAME:  ['4,roberto clemente bridge']
TRUE:  ['5,ptolemy xiii theos philopator', '5,cleopatra v']
PRED:  ['5,xiii theos philopator', '5,cleopatra v']
SAME:  ['5,cleopatra v']
PARTIAL:  ['5,cleopatra v']
TRUE:  ['6,li si']
PRED:  ['6,li si']
SAME:  ['6,li si']
TRUE:  ['7,pavel moroz', '7,yakov estrin']
PRED:  ['7,pavel moroz', '7,the yakov estrin']
SAME:  ['7,pavel moroz']
PARTIAL:  ['7,pavel moroz']
TRUE:  ['8,broadmeadows, victoria']
PRED:  ['8,broadmeadows', '8,victoria']
NONE_TRUE_MORE:  ['8,broadmeadows, victoria']
NONE

NONE_PRED_EQUAL:  ['']
TRUE:  ['179,screen actors guild life achievement award', '179,henry e. catto, jr.']
PRED:  ['179,henry e. catto', '179,jr.', '179,screen actors']
NONE_TRUE_MORE:  ['179,screen actors guild life achievement award', '179,henry e. catto, jr.']
NONE_PRED_MORE:  ['179,henry e. catto', '179,jr.', '179,screen actors']
TRUE:  ['180,pope gregory i']
PRED:  ['180,pope gregory']
SAME:  []
NONE_TRUE_EQUAL:  ['180,pope gregory i']
NONE_PRED_EQUAL:  ['180,pope gregory']
TRUE:  ['181,human rights party']
PRED:  ['181,cambodian']
SAME:  []
NONE_TRUE_EQUAL:  ['181,human rights party']
NONE_PRED_EQUAL:  ['181,cambodian']
TRUE:  ['182,czech extraliga']
PRED:  ['182,the czech extraliga']
SAME:  []
NONE_TRUE_EQUAL:  ['182,czech extraliga']
NONE_PRED_EQUAL:  ['182,the czech extraliga']
TRUE:  ['183,croix de guerre']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['183,croix de guerre']
NONE_PRED_EQUAL:  ['']
TRUE:  ['184,azam f.c.']
PRED:  ['184,azam f.c.']
SAME:  ['184,azam f.c.']
TRUE:  [

PRED:  ['309,papeete']
SAME:  ['309,papeete']
TRUE:  ['310,lake victoria']
PRED:  ['310, lake victoria']
SAME:  []
NONE_TRUE_EQUAL:  ['310,lake victoria']
NONE_PRED_EQUAL:  ['310, lake victoria']
TRUE:  ['311,domenico modugno']
PRED:  ['311,domenico modugno']
SAME:  ['311,domenico modugno']
TRUE:  ['312,grameen bank']
PRED:  ['312,grameen bank']
SAME:  ['312,grameen bank']
TRUE:  ['313,benjamin franklin terry', '313,us route 59']
PRED:  ['313,benjamin franklin terry', '313,us route 59']
SAME:  ['313,benjamin franklin terry', '313,us route 59']
TRUE:  ['314,james langer']
PRED:  ['314,james langer']
SAME:  ['314,james langer']
TRUE:  ['315,john trumbull', '315,a foreign policy']
PRED:  ['315,john trumbull']
LESS:  ['315,john trumbull']
TRUE:  ['316,peter-john vettese']
PRED:  ['316,peter-john vettese']
SAME:  ['316,peter-john vettese']
TRUE:  ['317,edwin catmull']
PRED:  ['317,edwin catmull']
SAME:  ['317,edwin catmull']
TRUE:  ['318,eric roth']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  [

PRED:  ['378,novelas']
SAME:  []
NONE_TRUE_EQUAL:  ['378,novelas ejemplares']
NONE_PRED_EQUAL:  ['378,novelas']
TRUE:  ['379,anne spielberg']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['379,anne spielberg']
NONE_PRED_EQUAL:  ['']
TRUE:  ['380,nbc', '380,frasier']
PRED:  ['380,nbc', '380,fraiser']
SAME:  ['380,nbc']
PARTIAL:  ['380,nbc']
TRUE:  ['381,washington station', '381,cta blue line']
PRED:  ['381,the cta blue line']
NONE_TRUE_LESS:  ['381,washington station', '381,cta blue line']
NONE_PRED_LESS:  ['381,the cta blue line']
TRUE:  ['382,janata party']
PRED:  ['382,one', '382,janata party']
MORE:  ['382,janata party']
TRUE:  ['383,ford theatre']
PRED:  ['383,ford theatre']
SAME:  ['383,ford theatre']
TRUE:  ['384,the adventures of mimi']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['384,the adventures of mimi']
NONE_PRED_EQUAL:  ['']
TRUE:  ['385,arlington county, virginia']
PRED:  ['385,arlington county', '385,virginia']
NONE_TRUE_MORE:  ['385,arlington county, virginia']
NONE_PRED_MORE

SAME:  []
NONE_TRUE_EQUAL:  ['510,kingston, ontario', '510,st lawrence river']
NONE_PRED_EQUAL:  ['510,kingston', '510,ontario']
TRUE:  ['511,mtr']
PRED:  ['511,mtr']
SAME:  ['511,mtr']
TRUE:  ['512,vestigial peter']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['512,vestigial peter']
NONE_PRED_EQUAL:  ['']
TRUE:  ['513,buccinidae']
PRED:  ['513,buccinidae']
SAME:  ['513,buccinidae']
TRUE:  ['514,channel 9 mcot hd', '514,ministry of information and communication technology']
PRED:  ['514,the ministry of information and communication technology (thailand', '514, of the channel 9 mcot hd']
SAME:  []
NONE_TRUE_EQUAL:  ['514,channel 9 mcot hd', '514,ministry of information and communication technology']
NONE_PRED_EQUAL:  ['514,the ministry of information and communication technology (thailand', '514, of the channel 9 mcot hd']
TRUE:  ['515,kumta']
PRED:  ['515,kumta']
SAME:  ['515,kumta']
TRUE:  ['516,mac os 9', '516,tiny tower']
PRED:  ['516,mac os 9', '516,tiny tower']
SAME:  ['516,mac os 9',

...

TRUE:  ['4082,zachary taylor', '4082,sixth infantry regiment of us']
PRED:  ['4082,zachary taylor', '4082,sixth', '4082,us']
MORE:  ['4082,zachary taylor']
TRUE:  ['4083,chicago bulls', '4083,phoenix suns']
PRED:  ['4083,chicago bulls', '4083,phoenix suns']
SAME:  ['4083,chicago bulls', '4083,phoenix suns']
TRUE:  ['4084,islam']
PRED:  ['4084,islam']
SAME:  ['4084,islam']
TRUE:  ['4085,timothy j. roemer']
PRED:  ['4085,timothy j. roemer']
SAME:  ['4085,timothy j. roemer']
TRUE:  ['4086,becket fund for religious liberty']
PRED:  ['4086,becket']
SAME:  []
NONE_TRUE_EQUAL:  ['4086,becket fund for religious liberty']
NONE_PRED_EQUAL:  ['4086,becket']
TRUE:  ['4087,tornado']
PRED:  ['4087,tornado']
SAME:  ['4087,tornado']
TRUE:  ['4088,henri maspero']
PRED:  ['4088,henri maspero']
SAME:  ['4088,henri maspero']
TRUE:  ['4089,software as a service']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['4089,software as a service']
NONE_PRED_EQUAL:  ['']
TRUE:  ['4090,remote desktop protocol']
PRED:  ['']

TRUE:  ['4283,pork']
PRED:  ['4283,one']
SAME:  []
NONE_TRUE_EQUAL:  ['4283,pork']
NONE_PRED_EQUAL:  ['4283,one']
TRUE:  ['4284,penn state nittany lions football']
PRED:  ['4284,penn state nittany lions']
SAME:  []
NONE_TRUE_EQUAL:  ['4284,penn state nittany lions football']
NONE_PRED_EQUAL:  ['4284,penn state nittany lions']
TRUE:  ['4285,sony corpn shareholders and subsidies']
PRED:  ['4285,sony corp']
SAME:  []
NONE_TRUE_EQUAL:  ['4285,sony corpn shareholders and subsidies']
NONE_PRED_EQUAL:  ['4285,sony corp']
TRUE:  ['4286,dan mica']
PRED:  ['4286,dan mica']
SAME:  ['4286,dan mica']
TRUE:  ['4287,gary portnoy', '4287,where everybody knows your name']
PRED:  ['4287,gary portnoy']
LESS:  ['4287,gary portnoy']
TRUE:  ['4288,rachel rasmussen']
PRED:  ['4288,rachel rasmussen']
SAME:  ['4288,rachel rasmussen']
TRUE:  ['4289,liliya lobanova', '4289,dofinivka estuary']
PRED:  ['4289, ', '4289,the liliya lobanova']
SAME:  []
NONE_TRUE_EQUAL:  ['4289,liliya lobanova', '4289,dofinivka estuar

PRED:  ['4490,massachusetts']
SAME:  []
NONE_TRUE_EQUAL:  ['4490,massachusetts house of representatives']
NONE_PRED_EQUAL:  ['4490,massachusetts']
TRUE:  ['4491,naval support activity naples']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['4491,naval support activity naples']
NONE_PRED_EQUAL:  ['']
TRUE:  ['4492,microsoft azure']
PRED:  ['4492,microsoft', '4492,one']
NONE_TRUE_MORE:  ['4492,microsoft azure']
NONE_PRED_MORE:  ['4492,microsoft', '4492,one']
TRUE:  ['4493,trinity house', '4493,royal society']
PRED:  ['4493,royal society', '4493,trinity house']
SAME:  ['4493,royal society', '4493,trinity house']
TRUE:  ['4494,williamsburg, kansas']
PRED:  ['4494,williamsburg', '4494,kansas']
NONE_TRUE_MORE:  ['4494,williamsburg, kansas']
NONE_PRED_MORE:  ['4494,williamsburg', '4494,kansas']
TRUE:  ['4495,ins viraat', '4495,daund junction railway station']
PRED:  ['4495,daund junction', '4495,ins']
SAME:  []
NONE_TRUE_EQUAL:  ['4495,ins viraat', '4495,daund junction railway station']
NONE_PRED_E

TRUE:  ['4699,cbs']
PRED:  ['4699,cbs']
SAME:  ['4699,cbs']
TRUE:  ['4700,massimo liverani', '4700,guido guerrini']
PRED:  ['4700,guido guerrini', '4700,massimo liverani']
SAME:  ['4700,guido guerrini', '4700,massimo liverani']
TRUE:  ['4701,california']
PRED:  ['4701,california']
SAME:  ['4701,california']
TRUE:  ['4702,world war ii']
PRED:  ['4702,one', '4702,world war ii']
MORE:  ['4702,world war ii']
TRUE:  ['4703,rev']
PRED:  ['4703,rev']
SAME:  ['4703,rev']
TRUE:  ['4704,erich bagge']
PRED:  ['4704,erich bagge']
SAME:  ['4704,erich bagge']
TRUE:  ['4705,japan']
PRED:  ['4705,japan']
SAME:  ['4705,japan']
TRUE:  ['4706,trn vit hng']
PRED:  ['4706,trn vit hng']
SAME:  ['4706,trn vit hng']
TRUE:  ['4707,albert kwesi ocran']
PRED:  ['4707,albert kwesi ocran']
SAME:  ['4707,albert kwesi ocran']
TRUE:  ['4708,sony financial']
PRED:  ['4708,sony financial']
SAME:  ['4708,sony financial']
TRUE:  ['4709,william l. dayton']
PRED:  ['4709,william l. dayton']
SAME:  ['4709,william l. dayton'

SAME:  ['4965,laurie johnson']
PARTIAL:  ['4965,laurie johnson']
TRUE:  ['4966,uss kittiwake', '4966,haight-ashbury']
PRED:  ['4966,haight-ashbury', '4966,uss kittiwake']
SAME:  ['4966,haight-ashbury', '4966,uss kittiwake']
TRUE:  ['4967,motorpsycho nitemare']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['4967,motorpsycho nitemare']
NONE_PRED_EQUAL:  ['']
TRUE:  ['4968,hbo', '4968,erik jendresen']
PRED:  ['4968,erik jendresen', '4968,hbo']
SAME:  ['4968,erik jendresen', '4968,hbo']
TRUE:  ['4969,battle']
PRED:  ['4969,battle of the bulge']
SAME:  []
NONE_TRUE_EQUAL:  ['4969,battle']
NONE_PRED_EQUAL:  ['4969,battle of the bulge']
TRUE:  ['4970,israel', '4970,colin powell']
PRED:  ['4970,colin powell']
LESS:  ['4970,colin powell']
TRUE:  ['4971,the ultimate fighter: team rousey vs. team tate']
PRED:  ['4971,the ultimate fighter', '4971,team tate']
NONE_TRUE_MORE:  ['4971,the ultimate fighter: team rousey vs. team tate']
NONE_PRED_MORE:  ['4971,the ultimate fighter', '4971,team tate']
TRUE:  

In [23]:
print(match)
print(partial)
print(less)
print(more)
print(none_match)
print(none_less)
print(none_more)
print(predicted)
print(predicted/len(real_entities))

2238
299
210
320
1322
198
413
4102
0.6175850647395363


In [24]:
### compare only md predictions

match, partial, less, more, none_match, none_less, none_more, predicted = compare_predictions(real, ner_md)
print(match)
print(partial)
print(less)
print(more)
print(none_match)
print(none_less)
print(none_more)
print(predicted)
print(predicted/len(real_entities))

TRUE:  ['0,bill finger']
PRED:  ['0,bill finger']
SAME:  ['0,bill finger']
TRUE:  ['1,selwyn lloyd', '1,winston churchill']
PRED:  ['1,selwyn lloyd']
LESS:  ['1,selwyn lloyd']
TRUE:  ['2,gestapo']
PRED:  ['2,gestapo']
SAME:  ['2,gestapo']
TRUE:  ['3,mumbai north']
PRED:  ['3,mumbai']
SAME:  []
NONE_TRUE_EQUAL:  ['3,mumbai north']
NONE_PRED_EQUAL:  ['3,mumbai']
TRUE:  ['4,roberto clemente bridge']
PRED:  ['4,roberto clemente bridge']
SAME:  ['4,roberto clemente bridge']
TRUE:  ['5,ptolemy xiii theos philopator', '5,cleopatra v']
PRED:  ['5,ptolemy xiii theos philopator', '5,cleopatra v']
SAME:  ['5,ptolemy xiii theos philopator', '5,cleopatra v']
TRUE:  ['6,li si']
PRED:  ['6,li si']
SAME:  ['6,li si']
TRUE:  ['7,pavel moroz', '7,yakov estrin']
PRED:  ['7,pavel moroz', '7,yakov estrin']
SAME:  ['7,pavel moroz', '7,yakov estrin']
TRUE:  ['8,broadmeadows, victoria']
PRED:  ['8,broadmeadows, victoria']
SAME:  ['8,broadmeadows, victoria']
TRUE:  ['9,asc creative services']
PRED:  ['9,asc cr

PRED:  ['164,josiah idowu-fearon', '164,damasucs']
SAME:  ['164,josiah idowu-fearon']
PARTIAL:  ['164,josiah idowu-fearon']
TRUE:  ['165,paramount television', '165,judy hart angelo']
PRED:  ['165,judy hart angelo', '165,paramount television']
SAME:  ['165,judy hart angelo', '165,paramount television']
TRUE:  ['166,caspian']
PRED:  ['166,caspian']
SAME:  ['166,caspian']
TRUE:  ['167,united states department of defense', '167,arlington county, virginia']
PRED:  ['167,arlington county', '167,virginia', '167,united states department of defense']
MORE:  ['167,united states department of defense']
TRUE:  ['168,thongsuk samdaengpan']
PRED:  ['168,thongsuk samdaengpan']
SAME:  ['168,thongsuk samdaengpan']
TRUE:  ['169,asus']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['169,asus']
NONE_PRED_EQUAL:  ['']
TRUE:  ['170,ncaa']
PRED:  ['170,ncaa']
SAME:  ['170,ncaa']
TRUE:  ['171,afrika baby bam', '171,queen latifah']
PRED:  ['171,queen latifah', '171,afrika', '171,baby bam']
MORE:  ['171,queen latifa

PRED:  ['298,bloomberg markets']
SAME:  ['298,bloomberg markets']
TRUE:  ['299,motor vehicles']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['299,motor vehicles']
NONE_PRED_EQUAL:  ['']
TRUE:  ['300,holmes colbert', '300,linda hogan']
PRED:  ['300,linda hogan', '300,holmes colbert']
SAME:  ['300,linda hogan', '300,holmes colbert']
TRUE:  ['301,jon speelman']
PRED:  ['301,jon speelman']
SAME:  ['301,jon speelman']
TRUE:  ['302,ella loves cole']
PRED:  ['302,ella loves cole']
SAME:  ['302,ella loves cole']
TRUE:  ['303,prime time entertainment network']
PRED:  ['303,prime time entertainment network']
SAME:  ['303,prime time entertainment network']
TRUE:  ['304,auto shankar']
PRED:  ['304,auto shankar']
SAME:  ['304,auto shankar']
TRUE:  ['305,burr truss']
PRED:  ['305,burr truss']
SAME:  ['305,burr truss']
TRUE:  ['306,tonle sap']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['306,tonle sap']
NONE_PRED_EQUAL:  ['']
TRUE:  ['307,cheryl teigs']
PRED:  ['307,cheryl teigs']
SAME:  ['307,cheryl teigs'

NONE_PRED_EQUAL:  ['371,gm']
TRUE:  ['372,texas']
PRED:  ['372,texas']
SAME:  ['372,texas']
TRUE:  ['373,counterpoint']
PRED:  ['373,counterpoint']
SAME:  ['373,counterpoint']
TRUE:  ['374,pc 9800']
PRED:  ['374,9800']
SAME:  []
NONE_TRUE_EQUAL:  ['374,pc 9800']
NONE_PRED_EQUAL:  ['374,9800']
TRUE:  ['375,anglo-frisian']
PRED:  ['375,anglo']
SAME:  []
NONE_TRUE_EQUAL:  ['375,anglo-frisian']
NONE_PRED_EQUAL:  ['375,anglo']
TRUE:  ['376,tony bennett']
PRED:  ['376,tony bennett']
SAME:  ['376,tony bennett']
TRUE:  ['377,charles willing byrd', '377,north bend, ohio']
PRED:  ['377,charles willing byrd', '377,north bend', '377,ohio']
MORE:  ['377,charles willing byrd']
TRUE:  ['378,novelas ejemplares']
PRED:  ['378,novelas']
SAME:  []
NONE_TRUE_EQUAL:  ['378,novelas ejemplares']
NONE_PRED_EQUAL:  ['378,novelas']
TRUE:  ['379,anne spielberg']
PRED:  ['379,anne spielberg']
SAME:  ['379,anne spielberg']
TRUE:  ['380,nbc', '380,frasier']
PRED:  ['380,nbc', '380,fraiser']
SAME:  ['380,nbc']
PARTI

TRUE:  ['439,us']
PRED:  ['439,us']
SAME:  ['439,us']
TRUE:  ['440,indian ocean', '440,zambia']
PRED:  ['440,indian ocean', '440,zambia']
SAME:  ['440,indian ocean', '440,zambia']
TRUE:  ['441,panathinaikos fc']
PRED:  ['441,panathinaikos fc']
SAME:  ['441,panathinaikos fc']
TRUE:  ['442,clamp']
PRED:  ['442,clamp']
SAME:  ['442,clamp']
TRUE:  ['443,raymond conner']
PRED:  ['443,raymond conner']
SAME:  ['443,raymond conner']
TRUE:  ['444,dominican republic']
PRED:  ['444,dominican republic']
SAME:  ['444,dominican republic']
TRUE:  ['445,charles journet', '445,elbieta czartoryska']
PRED:  ['445,the elbieta czartoryska', '445,the charles journet']
SAME:  []
NONE_TRUE_EQUAL:  ['445,charles journet', '445,elbieta czartoryska']
NONE_PRED_EQUAL:  ['445,the elbieta czartoryska', '445,the charles journet']
TRUE:  ['446,olin corporation']
PRED:  ['446,olin corporation']
SAME:  ['446,olin corporation']
TRUE:  ['447,judson huss', '447,philippe tesnire']
PRED:  ['447,philippe']
NONE_TRUE_LESS:  [

...

PRED:  ['4435,gemini', '4435,apollo 11']
SAME:  ['4435,apollo 11']
PARTIAL:  ['4435,apollo 11']
TRUE:  ['4436,joe mckeehen']
PRED:  ['4436,joe mckeehen']
SAME:  ['4436,joe mckeehen']
TRUE:  ['4437,us']
PRED:  ['4437,us']
SAME:  ['4437,us']
TRUE:  ['4438,lohan ratwatte']
PRED:  ['4438,lohan ratwatte']
SAME:  ['4438,lohan ratwatte']
TRUE:  ['4439,tillingdale']
PRED:  ['4439,tillingdale']
SAME:  ['4439,tillingdale']
TRUE:  ['4440,truth prevails']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['4440,truth prevails']
NONE_PRED_EQUAL:  ['']
TRUE:  ['4441,andrew w. lewis']
PRED:  ['4441,andrew w. lewis']
SAME:  ['4441,andrew w. lewis']
TRUE:  ['4442,erik bork', '4442,stephen ambrose']
PRED:  ['4442,stephen ambrose', '4442,erik bork']
SAME:  ['4442,stephen ambrose', '4442,erik bork']
TRUE:  ['4443,mark steven']
PRED:  ['4443,mark steven']
SAME:  ['4443,mark steven']
TRUE:  ['4444,chicago and northwestern transportation company']
PRED:  ['4444,chicago', '4444,northwestern transportation']
NONE_TRUE_M

SAME:  ['4573,england']
TRUE:  ['4574,massachusetts']
PRED:  ['4574,massachusetts']
SAME:  ['4574,massachusetts']
TRUE:  ['4575,charles willing', '4575,john scott']
PRED:  ['4575,charles willing', '4575,john scott']
SAME:  ['4575,charles willing', '4575,john scott']
TRUE:  ['4576,warner bros. television']
PRED:  ['4576,warner bros. television']
SAME:  ['4576,warner bros. television']
TRUE:  ['4577,egypt nanotechnology center']
PRED:  ['4577,egypt nanotechnology center']
SAME:  ['4577,egypt nanotechnology center']
TRUE:  ['4578,akio morita']
PRED:  ['4578,akio morita']
SAME:  ['4578,akio morita']
TRUE:  ['4579,blue lagoon island', '4579,bahamas']
PRED:  ['']
NONE_TRUE_LESS:  ['4579,blue lagoon island', '4579,bahamas']
NONE_PRED_LESS:  ['']
TRUE:  ['4580,rheinmetall mg 60']
PRED:  ['4580,rheinmetall mg 60']
SAME:  ['4580,rheinmetall mg 60']
TRUE:  ['4581,p fitzwallace', '4581,k fordice']
PRED:  ['4581,k fordice', '4581,fitzwallace']
SAME:  ['4581,k fordice']
PARTIAL:  ['4581,k fordice']


TRUE:  ['4718,denver']
PRED:  ['4718,denver']
SAME:  ['4718,denver']
TRUE:  ['4719,american english']
PRED:  ['4719,american', '4719,english']
NONE_TRUE_MORE:  ['4719,american english']
NONE_PRED_MORE:  ['4719,american', '4719,english']
TRUE:  ['4720,teriitaria ii']
PRED:  ['4720,teriitaria ii']
SAME:  ['4720,teriitaria ii']
TRUE:  ['4721,andrew schally', '4721,wtul']
PRED:  ['4721,schally', '4721,wtul']
SAME:  ['4721,wtul']
PARTIAL:  ['4721,wtul']
TRUE:  ['4722,exxv-tv']
PRED:  ['4722,exxv-tv']
SAME:  ['4722,exxv-tv']
TRUE:  ['4723,republican river']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['4723,republican river']
NONE_PRED_EQUAL:  ['']
TRUE:  ['4724,all i need is your sweet lovin']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['4724,all i need is your sweet lovin']
NONE_PRED_EQUAL:  ['']
TRUE:  ['4725,trinity school, brentwood', '4725,buckhurst hill county high school']
PRED:  ['4725,trinity school', '4725,brentwood', '4725,buckhurst hill county high school']
MORE:  ['4725,buckhurst hill

SAME:  ['4850,copley medal']
TRUE:  ['4851,south korean legislative election of 1992']
PRED:  ['4851,south korean', '4851,1992']
NONE_TRUE_MORE:  ['4851,south korean legislative election of 1992']
NONE_PRED_MORE:  ['4851,south korean', '4851,1992']
TRUE:  ['4852,gerald reaven', '4852,stanford cardinal']
PRED:  ['4852,stanford', '4852,gerald reaven']
SAME:  ['4852,gerald reaven']
PARTIAL:  ['4852,gerald reaven']
TRUE:  ['4853,mark davis', '4853,o.co colliseum']
PRED:  ['4853,mark davis', '4853,o.co colliseum']
SAME:  ['4853,mark davis', '4853,o.co colliseum']
TRUE:  ["4854,bbc's tv", '4854,the sarah jane adventures']
PRED:  ['4854,bbc', '4854,sarah jane']
SAME:  []
NONE_TRUE_EQUAL:  ["4854,bbc's tv", '4854,the sarah jane adventures']
NONE_PRED_EQUAL:  ['4854,bbc', '4854,sarah jane']
TRUE:  ['4855,autodromo enzo e dino ferrari']
PRED:  ['4855,first', '4855,autodromo enzo e dino ferrari']
MORE:  ['4855,autodromo enzo e dino ferrari']
TRUE:  ['4856,canada']
PRED:  ['4856,canada']
SAME:  ['

NONE_PRED_EQUAL:  ['4986,kerguelen', '4986,french']
TRUE:  ['4987,weber thompson']
PRED:  ['4987,weber thompson']
SAME:  ['4987,weber thompson']
TRUE:  ['4988,edith vonnegut']
PRED:  ['4988,edith vonnegut']
SAME:  ['4988,edith vonnegut']
TRUE:  ['4989,iyoas i']
PRED:  ['4989,iyoas']
SAME:  []
NONE_TRUE_EQUAL:  ['4989,iyoas i']
NONE_PRED_EQUAL:  ['4989,iyoas']
TRUE:  ['4990,wcw mayhem']
PRED:  ['4990,wcw mayhem']
SAME:  ['4990,wcw mayhem']
TRUE:  ['4991,rca records']
PRED:  ['4991,rca']
SAME:  []
NONE_TRUE_EQUAL:  ['4991,rca records']
NONE_PRED_EQUAL:  ['4991,rca']
TRUE:  ['4992,nick diaz', '4992,brazilian jiu-jitsu']
PRED:  ['4992,brazilian', '4992,nick diaz']
SAME:  ['4992,nick diaz']
PARTIAL:  ['4992,nick diaz']
TRUE:  ['4993,amstelveen']
PRED:  ['4993,amstelveen']
SAME:  ['4993,amstelveen']
TRUE:  ['4994,choctaw']
PRED:  ['4994,choctaw']
SAME:  ['4994,choctaw']
TRUE:  ['4995,larry j. franco']
PRED:  ['4995,larry j. franco']
SAME:  ['4995,larry j. franco']
TRUE:  ['4996,tnt', '4996,j

In [25]:
print(match)
print(partial)
print(less)
print(more)
print(none_match)
print(none_less)
print(none_more)
print(predicted)
print(predicted/len(real_entities))

2528
328
181
298
1144
137
384
4501
0.6776573321288768


In [26]:
### compare sm+md predictions

match, partial, less, more, none_match, none_less, none_more, predicted = compare_predictions(real, ner_sm_md)
print(match)
print(partial)
print(less)
print(more)
print(none_match)
print(none_less)
print(none_more)
print(predicted)
print(predicted/len(real_entities))

TRUE:  ['0,bill finger']
PRED:  ['0,bill finger']
SAME:  ['0,bill finger']
TRUE:  ['1,selwyn lloyd', '1,winston churchill']
PRED:  ['1,selwyn lloyd']
LESS:  ['1,selwyn lloyd']
TRUE:  ['2,gestapo']
PRED:  ['2,gestapo']
SAME:  ['2,gestapo']
TRUE:  ['3,mumbai north']
PRED:  ['3,mumbai']
SAME:  []
NONE_TRUE_EQUAL:  ['3,mumbai north']
NONE_PRED_EQUAL:  ['3,mumbai']
TRUE:  ['4,roberto clemente bridge']
PRED:  ['4,roberto clemente bridge']
SAME:  ['4,roberto clemente bridge']
TRUE:  ['5,ptolemy xiii theos philopator', '5,cleopatra v']
PRED:  ['5,xiii theos philopator', '5,cleopatra v']
SAME:  ['5,cleopatra v']
PARTIAL:  ['5,cleopatra v']
TRUE:  ['6,li si']
PRED:  ['6,li si']
SAME:  ['6,li si']
TRUE:  ['7,pavel moroz', '7,yakov estrin']
PRED:  ['7,pavel moroz', '7,the yakov estrin']
SAME:  ['7,pavel moroz']
PARTIAL:  ['7,pavel moroz']
TRUE:  ['8,broadmeadows, victoria']
PRED:  ['8,broadmeadows, victoria']
SAME:  ['8,broadmeadows, victoria']
TRUE:  ['9,asc creative services']
PRED:  ['9,asc cre

PARTIAL:  ['186,mausoleum of augustus']
TRUE:  ['187,1st free french division']
PRED:  ['187,1st free french division']
SAME:  ['187,1st free french division']
TRUE:  ['188,mediterranean sea']
PRED:  ['188,the mediterranean sea']
SAME:  []
NONE_TRUE_EQUAL:  ['188,mediterranean sea']
NONE_PRED_EQUAL:  ['188,the mediterranean sea']
TRUE:  ['189,zambia', '189,ikelenge district']
PRED:  ['189,zambia', '189,ikelenge']
SAME:  ['189,zambia']
PARTIAL:  ['189,zambia']
TRUE:  ['190,royal society']
PRED:  ['190,royal society']
SAME:  ['190,royal society']
TRUE:  ['191,acer nigrum']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['191,acer nigrum']
NONE_PRED_EQUAL:  ['']
TRUE:  ['192,filipinos']
PRED:  ['192,filipinos']
SAME:  ['192,filipinos']
TRUE:  ['193,neil brown', '193,andrew peacock']
PRED:  ['193,neil brown', '193,andrew peacock']
SAME:  ['193,neil brown', '193,andrew peacock']
TRUE:  ['194,wizards vs aliens', '194,into the dalek']
PRED:  ['194,into the dalek', '194,wizards']
SAME:  ['194,into th

PRED:  ['392,us']
SAME:  ['392,us']
TRUE:  ['393,random house']
PRED:  ['393,random house']
SAME:  ['393,random house']
TRUE:  ['394,lori black', '394,henry e. catto jr']
PRED:  ['394,lori black', '394,henry e. catto jr']
SAME:  ['394,lori black', '394,henry e. catto jr']
TRUE:  ['395,arizona state sun devils']
PRED:  ['395,arizona', '395,sun devils']
NONE_TRUE_MORE:  ['395,arizona state sun devils']
NONE_PRED_MORE:  ['395,arizona', '395,sun devils']
TRUE:  ['396,wyoming']
PRED:  ['396,wyoming']
SAME:  ['396,wyoming']
TRUE:  ['397,claiborne pell', '397,william luther hill']
PRED:  ['397,william luther hill', '397,claiborne pell']
SAME:  ['397,william luther hill', '397,claiborne pell']
TRUE:  ['398,ridley scott', '398,vangelis']
PRED:  ['398,ridley scott', '398,vangelis']
SAME:  ['398,ridley scott', '398,vangelis']
TRUE:  ['399,jeffrey werleman', '399,american mediterranean sea']
PRED:  ['399,jeffrey werleman', '399,american mediterranean sea']
SAME:  ['399,jeffrey werleman', '399,amer

NONE_PRED_EQUAL:  ['527,eo']
TRUE:  ['528,nairobi']
PRED:  ['528,nairobi']
SAME:  ['528,nairobi']
TRUE:  ['529,utopian and dystopian fiction']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['529,utopian and dystopian fiction']
NONE_PRED_EQUAL:  ['']
TRUE:  ['530,luke list', '530,k. terry dornbush']
PRED:  ['530,luke list', '530,k. terry dornbush']
SAME:  ['530,luke list', '530,k. terry dornbush']
TRUE:  ['531,philip novak']
PRED:  ['531,philip novak']
SAME:  ['531,philip novak']
TRUE:  ['532,westchester avenue', '532,the bronx']
PRED:  ['532,westchester avenue', '532,bronx']
SAME:  ['532,westchester avenue']
PARTIAL:  ['532,westchester avenue']
TRUE:  ['533,c__', '533,microsoft windows']
PRED:  ['533,c++', '533,microsoft', '533,windows']
NONE_TRUE_MORE:  ['533,c__', '533,microsoft windows']
NONE_PRED_MORE:  ['533,c++', '533,microsoft', '533,windows']
TRUE:  ['534,victor entertainment']
PRED:  ['534,the victor entertainment']
SAME:  []
NONE_TRUE_EQUAL:  ['534,victor entertainment']
NONE_PRED_

NONE_TRUE_EQUAL:  ['661,air interdiction']
NONE_PRED_EQUAL:  ['661,air']
TRUE:  ['662,the ultimate fighter: brazil 2']
PRED:  ['662,brazil', '662,2']
NONE_TRUE_MORE:  ['662,the ultimate fighter: brazil 2']
NONE_PRED_MORE:  ['662,brazil', '662,2']
TRUE:  ['663,tirana']
PRED:  ['663,tirana']
SAME:  ['663,tirana']
TRUE:  ['664,jon and vangelis']
PRED:  ['664,jon', '664,vangelis']
NONE_TRUE_MORE:  ['664,jon and vangelis']
NONE_PRED_MORE:  ['664,jon', '664,vangelis']
TRUE:  ['665,wcw mayhem']
PRED:  ['665,wcw']
SAME:  []
NONE_TRUE_EQUAL:  ['665,wcw mayhem']
NONE_PRED_EQUAL:  ['665,wcw']
TRUE:  ['666,marika gombitov']
PRED:  ['666,marika gombitov']
SAME:  ['666,marika gombitov']
TRUE:  ['667,9tv']
PRED:  ['667,9tv']
SAME:  ['667,9tv']
TRUE:  ['668,iraq', '668,army air corps']
PRED:  ['668,army air corps', '668,iraq']
SAME:  ['668,army air corps', '668,iraq']
TRUE:  ['669,north fort myers high school']
PRED:  ['669,north fort myers high school']
SAME:  ['669,north fort myers high school']
TRU

...

SAME:  ['4369,best mate']
TRUE:  ['4370,ozzy osbourne', '4370,kelly osbourne']
PRED:  ['4370,kelly osbourne & ozzy osbourne']
NONE_TRUE_LESS:  ['4370,ozzy osbourne', '4370,kelly osbourne']
NONE_PRED_LESS:  ['4370,kelly osbourne & ozzy osbourne']
TRUE:  ['4371,ontario']
PRED:  ['4371,ontario']
SAME:  ['4371,ontario']
TRUE:  ['4372,william anthony']
PRED:  ['4372,william anthony  ']
SAME:  []
NONE_TRUE_EQUAL:  ['4372,william anthony']
NONE_PRED_EQUAL:  ['4372,william anthony  ']
TRUE:  ['4373,us']
PRED:  ['4373,us']
SAME:  ['4373,us']
TRUE:  ['4374,city of miami cemetery']
PRED:  ['4374,miami cemetery']
SAME:  []
NONE_TRUE_EQUAL:  ['4374,city of miami cemetery']
NONE_PRED_EQUAL:  ['4374,miami cemetery']
TRUE:  ['4375,louisiana']
PRED:  ['4375,louisiana']
SAME:  ['4375,louisiana']
TRUE:  ['4376,neungin high school', '4376,buddhist school']
PRED:  ['4376,neungin high']
NONE_TRUE_LESS:  ['4376,neungin high school', '4376,buddhist school']
NONE_PRED_LESS:  ['4376,neungin high']
TRUE:  ['4377

SAME:  ['4508,grameen bank']
TRUE:  ['4509,earl b. ruth']
PRED:  ['4509,earl b. ruth']
SAME:  ['4509,earl b. ruth']
TRUE:  ['4510,mike kelly', '4510,tim mathieson']
PRED:  ['4510,the tim mathieson', '4510,the mike kelly', '4510,australian']
NONE_TRUE_MORE:  ['4510,mike kelly', '4510,tim mathieson']
NONE_PRED_MORE:  ['4510,the tim mathieson', '4510,the mike kelly', '4510,australian']
TRUE:  ['4511,katima mulilo bridge']
PRED:  ['4511,katima mulilo bridge']
SAME:  ['4511,katima mulilo bridge']
TRUE:  ['4512,black grape global']
PRED:  ['4512,black grape global']
SAME:  ['4512,black grape global']
TRUE:  ['4513,nintendo eshop']
PRED:  ['4513,nintendo eshop']
SAME:  ['4513,nintendo eshop']
TRUE:  ['4514,phil-ellena']
PRED:  ['4514,phil-ellena']
SAME:  ['4514,phil-ellena']
TRUE:  ['4515,lucius crus']
PRED:  ['4515,lucius crus']
SAME:  ['4515,lucius crus']
TRUE:  ['4516,david animle hanses']
PRED:  ['4516,david animle hanses']
SAME:  ['4516,david animle hanses']
TRUE:  ['4517,mastercard cent

TRUE:  ["4651,a christian turn'd turk"]
PRED:  ['4651,christian', '4651,turk']
NONE_TRUE_MORE:  ["4651,a christian turn'd turk"]
NONE_PRED_MORE:  ['4651,christian', '4651,turk']
TRUE:  ['4652,persona']
PRED:  ['4652,persona']
SAME:  ['4652,persona']
TRUE:  ['4653,mark stevens']
PRED:  ['4653,mark stevens']
SAME:  ['4653,mark stevens']
TRUE:  ['4654,josh groban']
PRED:  ['4654,josh groban']
SAME:  ['4654,josh groban']
TRUE:  ['4655,simon fuller', '4655,julian gingell']
PRED:  ['4655,julian gingell', '4655, simon fuller']
SAME:  ['4655,julian gingell']
PARTIAL:  ['4655,julian gingell']
TRUE:  ['4656,wbcq-fm']
PRED:  ['4656,wbcq-fm']
SAME:  ['4656,wbcq-fm']
TRUE:  ['4657,british columbia']
PRED:  ['4657,british columbia']
SAME:  ['4657,british columbia']
TRUE:  ['4658,latin', '4658,sovereign military order of malta']
PRED:  ['4658,latin', '4658,sovereign military order of malta']
SAME:  ['4658,latin', '4658,sovereign military order of malta']
TRUE:  ['4659,lake uniamsi']
PRED:  ['4659,lak

TRUE:  ['4788,into the dalek']
PRED:  ['4788,into the dalek']
SAME:  ['4788,into the dalek']
TRUE:  ['4789,citizen kane', '4789,gregg toland']
PRED:  ['4789,citizen kane', '4789,gregg toland']
SAME:  ['4789,citizen kane', '4789,gregg toland']
TRUE:  ['4790,bruce beutler']
PRED:  ['4790,bruce beutler']
SAME:  ['4790,bruce beutler']
TRUE:  ['4791,trump entertainment resorts', '4791,maryanne barry']
PRED:  ['4791,trump entertainment resorts', '4791,maryanne barry']
SAME:  ['4791,trump entertainment resorts', '4791,maryanne barry']
TRUE:  ['4792,barasat']
PRED:  ['4792,barasat']
SAME:  ['4792,barasat']
TRUE:  ['4793,alan freeman days', '4793,come on over']
PRED:  ['4793,alan freeman days']
LESS:  ['4793,alan freeman days']
TRUE:  ['4794,janice soprano']
PRED:  ['4794,janice soprano', '4794,soprano']
MORE:  ['4794,janice soprano']
TRUE:  ['4795,john muir', '4795,mt hoffmann']
PRED:  ['4795,first', '4795,john']
SAME:  []
NONE_TRUE_EQUAL:  ['4795,john muir', '4795,mt hoffmann']
NONE_PRED_EQUA

PARTIAL:  ['4926,krista kelly']
TRUE:  ['4927,judge advocate general corps']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['4927,judge advocate general corps']
NONE_PRED_EQUAL:  ['']
TRUE:  ['4928,pope pius xi']
PRED:  ['4928,pope']
SAME:  []
NONE_TRUE_EQUAL:  ['4928,pope pius xi']
NONE_PRED_EQUAL:  ['4928,pope']
TRUE:  ['4929,brown']
PRED:  ['4929,brown']
SAME:  ['4929,brown']
TRUE:  ['4930,solon spencer beman']
PRED:  ['4930,solon spencer beman']
SAME:  ['4930,solon spencer beman']
TRUE:  ['4931,1967 mexican grand prix', '4931,1971 us']
PRED:  ['4931,first', '4931,the 1967 mexican grand prix', '4931,1971', '4931,us']
NONE_TRUE_MORE:  ['4931,1967 mexican grand prix', '4931,1971 us']
NONE_PRED_MORE:  ['4931,first', '4931,the 1967 mexican grand prix', '4931,1971', '4931,us']
TRUE:  ['4932,sam sen railway station']
PRED:  ['4932,sam sen railway station']
SAME:  ['4932,sam sen railway station']
TRUE:  ['4933,newcastle']
PRED:  ['4933,newcastle']
SAME:  ['4933,newcastle']
TRUE:  ['4934,tehran']

In [27]:
print(match)
print(partial)
print(less)
print(more)
print(none_match)
print(none_less)
print(none_more)
print(predicted)
print(predicted/len(real_entities))

2511
352
187
274
1175
156
345
4446
0.6693766937669376


In [ ]:
# flair + spaCy

In [32]:
flair_spacy = ['']*n

for i in range(0, n):
    if len(ner_ner12[i])==len(ner_md[i]):
        if all(elem in ner_md[i] for elem in ner_ner12[i]) == True:
            flair_spacy[i] = ner_ner12[i]
        else:

            if ner_ner12[i] == ['']:
                flair_spacy[i] = ner_md[i]            
            else:
                flair_spacy[i] = ner_ner12[i]            
    elif len(ner_ner12[i])<len(ner_md[i]):
        if all(elem in ner_md[i] for elem in ner_ner12[i]) == True:
            flair_spacy[i] = ner_md[i]           
        else:
            if ner_ner12[i] == ['']:
                flair_spacy[i] = ner_md[i]            
            else:
                inter = list(set(ner_ner12[i]) & set(ner_md[i]))
                dif = list(set(ner_ner12[i]) - set(ner_md[i]))
                flair_spacy[i] = inter+dif
                
    elif len(ner_ner12[i])>len(ner_md[i]):
        if all(elem in ner_ner12[i] for elem in ner_md[i]) == True:
            flair_spacy[i] = ner_ner12[i]
        else:
            if ner_md[i] == ['']:
                flair_spacy[i] = ner_ner12[i]
            else:
                inter = list(set(ner_ner12[i]) & set(ner_md[i]))
                dif = list(set(ner_md[i]) - set(ner_ner12[i]))
                flair_spacy[i] = inter+dif

[['0,Bill Finger'], ['1,winston churchill', '1,Selwyn Lloyd'], ['2,Gestapo'], ['3,Mumbai North'], ['4,Roberto Clemente Bridge'], ['5,Theos Philopator', '5,Cleopatra'], ['6,Li Si'], ['7,PAvel Moroz', '7,Yakov Estrin'], ['8,Broadmeadows, Victoria'], ['9,ASC Creative Services'], ['10,Ernest Rutherford', '10,Charles Drummond Ellis'], ['11,REP Parasol'], ["12,Monroe Carell Jr. Children's Hospital", '12,Vanderbilt', '12,Duncan U. Fletcher'], ['13,Nader Guirat,', '13,Josef Johansson'], ['14,sarah jane'], ['15,HBO'], ['16,Nikolai Morozov', '16,Stanislav Morozov'], ['17,Abhijit Kunte', '17,Kasparov'], ['18,MSX Basics'], ['19,seasons', '19,Ronaldo'], ['20,SamurAbsheron', '20,Anar Salmanov'], ['21,2nd Foreign Infantry Regiment'], ['22,SR'], ['23,Kevin Jonas', '23,Joe Jonas'], ['24,Timm Gunn', '24,Sunrise', '24,HIMYM'], ['25,O.co Coliseum'], ['26,Rishkiesh'], ['27,america'], ['28,PhDs', '28,National Medal of Science'], ['29,Middlesbrough F.C.', '29,Aston Villa'], ['30,William Harper.'], ['31,PCL']

In [33]:
### compare flair (ner+ner12) + spaCy(md) predictions

match, partial, less, more, none_match, none_less, none_more, predicted = compare_predictions(real, flair_spacy)
print(match)
print(partial)
print(less)
print(more)
print(none_match)
print(none_less)
print(none_more)
print(predicted)
print(predicted/len(real_entities))

TRUE:  ['0,bill finger']
PRED:  ['0,bill finger']
SAME:  ['0,bill finger']
TRUE:  ['1,selwyn lloyd', '1,winston churchill']
PRED:  ['1,winston churchill', '1,selwyn lloyd']
SAME:  ['1,winston churchill', '1,selwyn lloyd']
TRUE:  ['2,gestapo']
PRED:  ['2,gestapo']
SAME:  ['2,gestapo']
TRUE:  ['3,mumbai north']
PRED:  ['3,mumbai north']
SAME:  ['3,mumbai north']
TRUE:  ['4,roberto clemente bridge']
PRED:  ['4,roberto clemente bridge']
SAME:  ['4,roberto clemente bridge']
TRUE:  ['5,ptolemy xiii theos philopator', '5,cleopatra v']
PRED:  ['5,theos philopator', '5,cleopatra']
SAME:  []
NONE_TRUE_EQUAL:  ['5,ptolemy xiii theos philopator', '5,cleopatra v']
NONE_PRED_EQUAL:  ['5,theos philopator', '5,cleopatra']
TRUE:  ['6,li si']
PRED:  ['6,li si']
SAME:  ['6,li si']
TRUE:  ['7,pavel moroz', '7,yakov estrin']
PRED:  ['7,pavel moroz', '7,yakov estrin']
SAME:  ['7,pavel moroz', '7,yakov estrin']
TRUE:  ['8,broadmeadows, victoria']
PRED:  ['8,broadmeadows, victoria']
SAME:  ['8,broadmeadows, v

NONE_PRED_MORE:  ['268,alpine', "268,winter olympics men's"]
TRUE:  ['269,virgin radio']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['269,virgin radio']
NONE_PRED_EQUAL:  ['']
TRUE:  ['270,tani university']
PRED:  ['270,tani university']
SAME:  ['270,tani university']
TRUE:  ['271,dowra']
PRED:  ['271,dowra']
SAME:  ['271,dowra']
TRUE:  ['272,pittsburgh pirates']
PRED:  ['272,pittsburgh pirates']
SAME:  ['272,pittsburgh pirates']
TRUE:  ['273,simon fuller']
PRED:  ['273,simon fuller']
SAME:  ['273,simon fuller']
TRUE:  ['274,the producers', '274,next thing']
PRED:  ['274,next thing', '274,the producers']
SAME:  ['274,next thing', '274,the producers']
TRUE:  ['275,henry ford', '275,ford model c']
PRED:  ['275,henry ford', '275,model c']
SAME:  ['275,henry ford']
PARTIAL:  ['275,henry ford']
TRUE:  ["276,dragons' den"]
PRED:  ["276,dragons' den"]
SAME:  ["276,dragons' den"]
TRUE:  ['277,joseph case high school', '277,providence chapel, charlwood']
PRED:  ['277,the providence chapel, charlwo

TRUE:  ['428,raptors 905']
PRED:  ['428,raptors 905']
SAME:  ['428,raptors 905']
TRUE:  ['429,carmel winery']
PRED:  ['429,carmel winery']
SAME:  ['429,carmel winery']
TRUE:  ['430,irishmen']
PRED:  ['430,irishmen']
SAME:  ['430,irishmen']
TRUE:  ['431,nea smyrni']
PRED:  ['431,nea smyrni']
SAME:  ['431,nea smyrni']
TRUE:  ['432,ingmar bergman', "432,james o'brien"]
PRED:  ['432,ingmar bergman', "432,james o'brien"]
SAME:  ['432,ingmar bergman', "432,james o'brien"]
TRUE:  ['433,colonel', '433,militia']
PRED:  ['433,militia']
LESS:  ['433,militia']
TRUE:  ['434,toronto']
PRED:  ['434,toronto']
SAME:  ['434,toronto']
TRUE:  ['435,mike salmon']
PRED:  ['435,mike salmon']
SAME:  ['435,mike salmon']
TRUE:  ['436,julius caesar']
PRED:  ['436,kingship', '436,julius caesar']
MORE:  ['436,julius caesar']
TRUE:  ['437,mary poppins']
PRED:  ['437,mary poppins']
SAME:  ['437,mary poppins']
TRUE:  ['438,bob macmillan', '438,mikhail pashnin']
PRED:  ['438,bob macmillan', '438,mikhail pashnin']
SAME

PRED:  ["571,president's", '571,brockport golden eagles']
MORE:  ['571,brockport golden eagles']
TRUE:  ['572,bruce bochy', '572,ramiro pea']
PRED:  ['572,ramiro pea', '572,bruce bochy']
SAME:  ['572,ramiro pea', '572,bruce bochy']
TRUE:  ['573,tnt', '573,jeff conaway']
PRED:  ['573,jeff conaway', '573,tnt']
SAME:  ['573,jeff conaway', '573,tnt']
TRUE:  ['574,aegean sea']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['574,aegean sea']
NONE_PRED_EQUAL:  ['']
TRUE:  ['575,john tory']
PRED:  ['575,john tory']
SAME:  ['575,john tory']
TRUE:  ['576,north caucasus railway']
PRED:  ['576,north caucasus']
SAME:  []
NONE_TRUE_EQUAL:  ['576,north caucasus railway']
NONE_PRED_EQUAL:  ['576,north caucasus']
TRUE:  ['577,etienne bieler']
PRED:  ['577,etienne bieler\\xc9tienne_bi\\xe9ler']
SAME:  []
NONE_TRUE_EQUAL:  ['577,etienne bieler']
NONE_PRED_EQUAL:  ['577,etienne bieler\\xc9tienne_bi\\xe9ler']
TRUE:  ['578,london']
PRED:  ['578,london']
SAME:  ['578,london']
TRUE:  ['579,muhammad yunus']
PRED:  [

SAME:  ['775,winston bryant', '775,joseph stiglitz']
TRUE:  ['776,jeevan mrityu']
PRED:  ['776,jeevan mrityu']
SAME:  ['776,jeevan mrityu']
TRUE:  ['777,nathaniel p. hill']
PRED:  ['777,nathaniel p. hill']
SAME:  ['777,nathaniel p. hill']
TRUE:  ['778,tom maniatis', '778,james still']
PRED:  ['778,james still', '778,tom maniatis']
SAME:  ['778,james still', '778,tom maniatis']
TRUE:  ['779,becket fund for religious liberty']
PRED:  ['779,becket']
SAME:  []
NONE_TRUE_EQUAL:  ['779,becket fund for religious liberty']
NONE_PRED_EQUAL:  ['779,becket']
TRUE:  ['780,joe purcell', '780,winston bryant']
PRED:  ['780,winston bryant', '780,joe purcell']
SAME:  ['780,winston bryant', '780,joe purcell']
TRUE:  ['781,scuderia ferrari']
PRED:  ['781,scuderia ferrari,']
SAME:  []
NONE_TRUE_EQUAL:  ['781,scuderia ferrari']
NONE_PRED_EQUAL:  ['781,scuderia ferrari,']
TRUE:  ['782,olivier boulay']
PRED:  ['782,olivier boulay']
SAME:  ['782,olivier boulay']
TRUE:  ['783,mary poppins (musical)']
PRED:  ['

...

TRUE:  ['3943,san diego comic-con international']
PRED:  ['3943,san diego comic-con international']
SAME:  ['3943,san diego comic-con international']
TRUE:  ['3944,klang valley']
PRED:  ['3944,klang valley']
SAME:  ['3944,klang valley']
TRUE:  ['3945,norfolk southern railway']
PRED:  ['3945,norfolk southern railway']
SAME:  ['3945,norfolk southern railway']
TRUE:  ['3946,blanche bruce', '3946,george arceneaux, jr.']
PRED:  ['3946,george arceneaux, jr.', '3946,blanche bruce']
SAME:  ['3946,george arceneaux, jr.', '3946,blanche bruce']
TRUE:  ['3947,chaudhary devi lal']
PRED:  ['3947,chaudhary devi lal']
SAME:  ['3947,chaudhary devi lal']
TRUE:  ['3948,rhodes-haverty building']
PRED:  ['3948,rhodes-haverty building']
SAME:  ['3948,rhodes-haverty building']
TRUE:  ['3949,ptolemy xiv of egypt']
PRED:  ['3949,ptolemy xiv', '3949,egypt']
NONE_TRUE_MORE:  ['3949,ptolemy xiv of egypt']
NONE_PRED_MORE:  ['3949,ptolemy xiv', '3949,egypt']
TRUE:  ['3950,martin molony', '3950,best mate']
PRED:  ['

NONE_PRED_EQUAL:  ['4161,ohio']
TRUE:  ['4162,american boulevard']
PRED:  ['4162,american boulevard (metro transit']
SAME:  []
NONE_TRUE_EQUAL:  ['4162,american boulevard']
NONE_PRED_EQUAL:  ['4162,american boulevard (metro transit']
TRUE:  ['4163,michigan wolverines', '4163,chicago bears']
PRED:  ['4163,michigan wolverines', '4163,chicago bears']
SAME:  ['4163,michigan wolverines', '4163,chicago bears']
TRUE:  ['4164,liverpool playhouse']
PRED:  ['4164,liverpool playhouse']
SAME:  ['4164,liverpool playhouse']
TRUE:  ['4165,bangladesh']
PRED:  ['4165,one', '4165,bangladesh']
MORE:  ['4165,bangladesh']
TRUE:  ['4166,torrey pines gliderport']
PRED:  ['4166,torrey pines gliderport']
SAME:  ['4166,torrey pines gliderport']
TRUE:  ['4167,deion sanders']
PRED:  ['4167,deion sanders']
SAME:  ['4167,deion sanders']
TRUE:  ['4168,lori black', '4168,adrian a. basora']
PRED:  ['4168,adrian a. basora', '4168,lori black']
SAME:  ['4168,adrian a. basora', '4168,lori black']
TRUE:  ['4169,michael hou

SAME:  ['4381,national collegiate athletic association', '4381,nicholas s. zeppos']
TRUE:  ['4382,harry harlow']
PRED:  ['4382,phd', '4382,harry harlow']
MORE:  ['4382,harry harlow']
TRUE:  ['4383,pierre r. graham', '4383,william french smith']
PRED:  ['4383,william french smith', '4383,pierre r. graham']
SAME:  ['4383,william french smith', '4383,pierre r. graham']
TRUE:  ['4384,singapore', '4384,pulau ubin']
PRED:  ['4384,pulau ubin', '4384,singapore']
SAME:  ['4384,pulau ubin', '4384,singapore']
TRUE:  ['4385,nyc']
PRED:  ['4385,nyc']
SAME:  ['4385,nyc']
TRUE:  ['4386,inyo national forest']
PRED:  ['4386,inyo national forest']
SAME:  ['4386,inyo national forest']
TRUE:  ['4387,john paul ii']
PRED:  ['4387,john paul ii']
SAME:  ['4387,john paul ii']
TRUE:  ['4388,heathsville, virginia']
PRED:  ['4388,heathsville, virginia']
SAME:  ['4388,heathsville, virginia']
TRUE:  ['4389,sverre krogh sundbo', '4389,havard vad petersson']
PRED:  ['4389,sverre krogh sundbo']
LESS:  ['4389,sverre kr

LESS:  ['4604,karl ewald hasse']
TRUE:  ['4605,sierra nevada']
PRED:  ['4605,sierra nevada']
SAME:  ['4605,sierra nevada']
TRUE:  ['4606,animal']
PRED:  ['4606,animal kingdom']
SAME:  []
NONE_TRUE_EQUAL:  ['4606,animal']
NONE_PRED_EQUAL:  ['4606,animal kingdom']
TRUE:  ['4607,pau broca']
PRED:  ['4607,one', '4607,pau broca']
MORE:  ['4607,pau broca']
TRUE:  ['4608,chordate']
PRED:  ['4608,chordate']
SAME:  ['4608,chordate']
TRUE:  ['4609,sony', '4609,asus']
PRED:  ['4609,asus', '4609,sony']
SAME:  ['4609,asus', '4609,sony']
TRUE:  ['4610,country music']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['4610,country music']
NONE_PRED_EQUAL:  ['']
TRUE:  ['4611,instagram']
PRED:  ['']
SAME:  []
NONE_TRUE_EQUAL:  ['4611,instagram']
NONE_PRED_EQUAL:  ['']
TRUE:  ['4612,daniel gibson']
PRED:  ['4612,daniel gibson']
SAME:  ['4612,daniel gibson']
TRUE:  ['4613,homestead grays']
PRED:  ['4613,homestead grays']
SAME:  ['4613,homestead grays']
TRUE:  ['4614,harvard-westlake school', '4614,henry e. catto

PRED:  ['4818,christen-democratisch en vlaams']
SAME:  ['4818,christen-democratisch en vlaams']
TRUE:  ['4819,juno i']
PRED:  ['4819,juno i']
SAME:  ['4819,juno i']
TRUE:  ['4820,google web toolkit', '4820,playn']
PRED:  ['4820,playn']
LESS:  ['4820,playn']
TRUE:  ['4821,steven moffat']
PRED:  ['4821,steven moffat']
SAME:  ['4821,steven moffat']
TRUE:  ['4822,bbc two']
PRED:  ['4822,bbc two']
SAME:  ['4822,bbc two']
TRUE:  ['4823,tupelo mississippi']
PRED:  ['4823,tupelo mississippi']
SAME:  ['4823,tupelo mississippi']
TRUE:  ['4824,walt disney records']
PRED:  ['4824,walt disney records']
SAME:  ['4824,walt disney records']
TRUE:  ['4825,cohoes, ny']
PRED:  ['4825,cohoes', '4825,ny']
NONE_TRUE_MORE:  ['4825,cohoes, ny']
NONE_PRED_MORE:  ['4825,cohoes', '4825,ny']
TRUE:  ['4826,william campbell', '4826,columbia university']
PRED:  ['4826,columbia university', '4826,william campbell']
SAME:  ['4826,columbia university', '4826,william campbell']
TRUE:  ['4827,maharashtran']
PRED:  ['4827

In [34]:
print(match)
print(partial)
print(less)
print(more)
print(none_match)
print(none_less)
print(none_more)
print(predicted)
print(predicted/len(real_entities))

3190
330
135
229
872
110
134
5206
0.7838000602228244
